In [1]:
#Test
import sys
sys.path.insert(0, '../script/')

import torch
import torch.autograd as autograd
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable

import glob
import time
import numpy as np

import preprocess
X,Y = preprocess.load_data()

In [2]:
print("Finished cleaning")
def charToNumber(c):
    n = ord(c)
    if(n>=3585 and n<=3673):
        return n-3585
    else:
        return 89 #For Other

def prepare_sequence(sentence):
    idxs = [charToNumber(c) for c in sentence]
    tensor = torch.LongTensor(idxs)
    return Variable(tensor)
prepare_sequence("ทดสอบ")

Finished cleaning


Variable containing:
 22
 19
 41
 44
 25
[torch.LongTensor of size 5]

In [3]:
#Initiate parameters
emb_size = 32
nhidden = 300
nlayers = 5
dropout = 0.5
rnn_type = 'LSTM'
n_char = 90
clip = 0.25
nout = 1

In [4]:
class BiLSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, n_layers,char_size, tagset_size):
        super(BiLSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers

        self.word_embeddings = nn.Embedding(char_size, embedding_dim)
        
        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim//2,num_layers = n_layers,bidirectional=True)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # Before we've done anything, we dont have any hidden state.
        # Refer to the Pytorch documentation to see exactly
        # why they have this dimensionality.
        # The axes semantics are (num_layers, minibatch_size, hidden_dim)
        return (autograd.Variable(torch.zeros(self.n_layers*2, 1, self.hidden_dim//2)),
                autograd.Variable(torch.zeros(self.n_layers*2, 1, self.hidden_dim//2)))

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, self.hidden = self.lstm(
            embeds.view(len(sentence), 1, -1), self.hidden)
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.softmax(tag_space)
        return tag_scores


In [7]:
model = BiLSTMTagger(emb_size,nhidden,nlayers,n_char, nout)
loss_function = nn.BCELoss()
#loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01)

In [ ]:
epochs = 20
printEvery = 1000
i=0
j=0
for i in range(epochs):
    print("Starting epochs %d..."%i)
    start_time = time.time()
    shuffled_indexed = np.random.permutation(len(X))
    print(X.shape)
    print(Y.shape)
    for sentence, tags in zip(X[shuffled_indexed],Y[shuffled_indexed]):
        piped = preprocess.create_pipe(sentence,tags)
        if(piped):
            j+=1
            if(j%printEvery==0): print(piped)
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            model.zero_grad()

            # Also, we need to clear out the hidden state of the LSTM,
            # detaching it from its history on the last instance.
            model.hidden = model.init_hidden()

            # Step 2. Get our inputs ready for the network, that is, turn them into
            # Variables of word indices.
            sentence_in = prepare_sequence(sentence)
            #targets = prepare_sequence(tags, tag_to_ix)

            # Step 3. Run our forward pass.
            tag_scores = model(sentence_in)
            targets = Variable(torch.FloatTensor([int(i) for i in tags])).view(-1,1)


            pred_string = "".join([str(int(i[0])) for i in tag_scores.data.numpy()])
            if(j%printEvery==0): 
                print("Pred: ",end='')
                print(preprocess.create_pipe(sentence,pred_string))

            # Step 4. Compute the loss, gradients, and update the parameters by
            #  calling optimizer.step()
            loss = loss_function(tag_scores, targets)
            print("Loss: ",loss.data.numpy()[0],end='\r')
            loss.backward()
            optimizer.step()
        else:
            print("Skipping")
        
    print("Epochs: %d"%i)
    print("Done!")
    print("--- %s seconds ---" % (time.time() - start_time))
    print("Loss: %f"%(loss.data.numpy()[0]))
    train_loss.append(loss)
    print()

Starting epochs 0...
(21589,)
(21589,)
19ss:  19.0654
20
+++++++++++++++++++
00000000000000000001
Skipping
วลา|มี|ปัญหา|เกี่ยว|กับ|การ|ปฏิรูป|การ|เมือง|ขึ้น|มา|ครา|ใด| |ก็|จะ|มี|ผู้|ที่|เป็น|นัก|กฎหมาย|มหาชน|และ|รวม|ถึง|ผู้|ที่|เข้าใจ|ว
Pred: |ว|ล|า|ม|ี|ป|ั|ญ|ห|า|เ|ก|ี|่|ย|ว|ก|ั|บ|ก|า|ร|ป|ฏ|ิ|ร|ู|ป|ก|า|ร|เ|ม|ื|อ|ง|ข|ึ|้|น|ม|า|ค|ร|า|ใ|ด| |ก|็|จ|ะ|ม|ี|ผ|ู|้|ท|ี|่|เ|ป|็|น|น|ั|ก|ก|ฎ|ห|ม|า|ย|ม|ห|า|ช|น|แ|ล|ะ|ร|ว|ม|ถ|ึ|ง|ผ|ู|้|ท|ี|่|เ|ข|้|า|ใ|จ|ว
|แต่|ผู้|ผูกขาด|ความ|รุนแรง|ก็|ไม่|สามารถ|จำกัด|วง|มิ|ให้|ความ|รุนแรง|ระบาด|ไป|ยัง|ส่วน|อื่น|ๆ| |ของ|สังคม|ได้| |ฉะนั้น|สังคม|จึ
Pred: |แ|ต|่|ผ|ู|้|ผ|ู|ก|ข|า|ด|ค|ว|า|ม|ร|ุ|น|แ|ร|ง|ก|็|ไ|ม|่|ส|า|ม|า|ร|ถ|จ|ำ|ก|ั|ด|ว|ง|ม|ิ|ใ|ห|้|ค|ว|า|ม|ร|ุ|น|แ|ร|ง|ร|ะ|บ|า|ด|ไ|ป|ย|ั|ง|ส|่|ว|น|อ|ื|่|น|ๆ| |ข|อ|ง|ส|ั|ง|ค|ม|ไ|ด|้| |ฉ|ะ|น|ั|้|น|ส|ั|ง|ค|ม|จ|ึ
